<a href="https://colab.research.google.com/github/its-Kumar/Deep_Learning/blob/master/its-Kumar/1_Artificial_Neural_Networks/Evaluating_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Evaluating the ANN

## Part 1 - Data Preprocessing


### Importing the libraries


In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
tf.__version__

'2.2.0'

### Importing the dataset


In [2]:
dataset = pd.read_csv('Churn_Modelling.csv')
dataset

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [3]:
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values
X

array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object)

### Encoding categorical data



In [4]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [5]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
ct =  ColumnTransformer([('encoder', OneHotEncoder(), [1])],
                        remainder='passthrough')
X = ct.fit_transform(X)
X[0]

array([1.0, 0.0, 0.0, 619, 0, 42, 2, 0.0, 1, 1, 1, 101348.88],
      dtype=object)

### Splitting the dataset into the Training set and Test set


In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=0)

### Feature Scaling


In [7]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Evaluating, improving and tuning the ANN

### Evaluating the ANN


In [8]:
def build_classifier():
    classifier = keras.models.Sequential()
    classifier.add(keras.layers.Dense(units=8, activation='relu'))
    classifier.add(keras.layers.Dense(units=8, activation='relu'))
    classifier.add(keras.layers.Dense(units=1, activation='sigmoid'))
    classifier.compile(optimizer='adam', loss='binary_crossentropy',
                       metrics = ['accuracy'])
    return classifier

In [9]:
from keras.wrappers.scikit_learn import KerasClassifier
classifier = KerasClassifier(build_fn=build_classifier,
                             batch_size=25,
                             nb_epoch=100)

Using TensorFlow backend.


In [11]:
classifier.fit(X_train, y_train)

320/320 [==============================] - 0s 1ms/step - loss: 0.5647 - accuracy: 0.7211


In [13]:
y_pred = classifier.predict(X_test) > 0.5

In [16]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[1565,   30],
       [ 375,   30]])

In [17]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator=classifier,
                             X=X_train,
                             y=y_train,
                             cv=10)
print(accuracies.mean())
print(accuracies.std())

32/32 [==============================] - 0s 966us/step - loss: 0.4656 - accuracy: 0.7950
0.7960000038146973
0.010365206830375636


### improving the ANN

dropout regularization to reduce overfitting if needed

### Tuning the ANN


In [18]:
def build_classifier(optimizer):
    classifier = keras.models.Sequential()
    classifier.add(keras.layers.Dense(units=8, activation='relu'))
    classifier.add(keras.layers.Dense(units=8, activation='relu'))
    classifier.add(keras.layers.Dense(units=1, activation='sigmoid'))
    classifier.compile(optimizer=optimizer, loss='binary_crossentropy',
                       metrics = ['accuracy'])
    return classifier

In [20]:
from sklearn.model_selection import GridSearchCV
classifier = KerasClassifier(build_fn=build_classifier)
parameters = {
    'batch_size': [5, 10, 25, 32],
    'nb_epoch': [10, 50, 100, 300, 500],
    'optimizer': ['adam', 'rmsprop']
    }
grid_search = GridSearchCV(estimator=classifier,
                           param_grid=parameters,
                           scoring='accuracy',
                           cv=10)

In [21]:
grid_search = grid_search.fit(X_train, y_train)
best_param = grid_search.best_params_
best_acc = grid_search.best_score_

1600/1600 [==============================] - 2s 1ms/step - loss: 0.4746 - accuracy: 0.7886


In [22]:
print(best_param)

{'batch_size': 5, 'nb_epoch': 10, 'optimizer': 'adam'}


In [23]:
print(best_acc)

0.8106250000000002


## Making best Model

In [27]:
best_classifier = build_classifier(best_param['optimizer'])
best_classifier.fit(X_train, y_train,
                    batch_size=best_param['batch_size'],
                    epochs=best_param['nb_epoch'])

Epoch 1/10
1600/1600 [==============================] - 2s 1ms/step - loss: 0.5083 - accuracy: 0.7707
Epoch 2/10
1600/1600 [==============================] - 2s 1ms/step - loss: 0.4025 - accuracy: 0.8259
Epoch 3/10
1600/1600 [==============================] - 2s 1ms/step - loss: 0.3714 - accuracy: 0.8414
Epoch 4/10
1600/1600 [==============================] - 2s 1ms/step - loss: 0.3628 - accuracy: 0.8491
Epoch 5/10
1600/1600 [==============================] - 2s 1ms/step - loss: 0.3571 - accuracy: 0.8536
Epoch 6/10
1600/1600 [==============================] - 2s 1ms/step - loss: 0.3529 - accuracy: 0.8554
Epoch 7/10
1600/1600 [==============================] - 2s 1ms/step - loss: 0.3495 - accuracy: 0.8556
Epoch 8/10
1600/1600 [==============================] - 2s 1ms/step - loss: 0.3459 - accuracy: 0.8575
Epoch 9/10
1600/1600 [==============================] - 2s 1ms/step - loss: 0.3445 - accuracy: 0.8580
Epoch 10/10
1600/1600 [==============================] - 2s 1ms/step - loss: 0.343

In [28]:
y_pred = best_classifier.predict(X_test) > 0.5

In [29]:
confusion_matrix(y_test, y_pred)

array([[1540,   55],
       [ 212,  193]])

In [32]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.8665